In [2]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.8


In [3]:
from classifier.create_ws import AZHelper
ws = AZHelper.load_ws()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Found the config file in: /home/aaditya/PycharmProjects/Codefundopp/classifier/aml_config/config.json


Workspace name: pyt1
Azure region: eastus2
Subscription id: af7772d6-3275-4058-8d9e-7640742004e3
Resource group: Codefundopp


In [4]:
%%time
compute_target = AZHelper.load_cp(ws)

found compute target. just use it. gpucluster


{'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-01-21T19:13:53.144000+00:00', 'creationTime': '2019-01-21T18:11:51.838081+00:00', 'currentNodeCount': 0, 'errors': None, 'modifiedTime': '2019-01-21T18:12:14.840387+00:00', 'nodeStateCounts': {'idleNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0, 'preparingNodeCount': 0, 'runningNodeCount': 0, 'unusableNodeCount': 0}, 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'targetNodeCount': 0, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}
CPU times: user 64.4 ms, sys: 3.86 ms, total: 68.2 ms
Wall time: 3.57 s


In [5]:
import os

project_folder = './pytorch-distr-hvd'
os.makedirs(project_folder, exist_ok=True)

In [6]:
import shutil

shutil.copy('classifier/train_on_azure.py', project_folder)
shutil.copy('classifier/dataset.py', project_folder)
shutil.copy('classifier/xception.py', project_folder)

'./pytorch-distr-hvd/xception.py'

In [7]:
from azureml.core import Experiment

experiment_name = 'pytorch-distr-hvd'
experiment = Experiment(ws, name=experiment_name)

In [8]:
ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)

AzureBlob pyt1storagekitusvbf azureml-blobstore-7108e7e0-f9a2-45aa-99f1-27e69744c740


In [ ]:
ds.upload(src_dir='data_wf', target_path='data_wf', overwrite=True, show_progress=False)


In [52]:
ds.upload(src_dir='classifier/weights', target_path='weights', overwrite=True, show_progress=True)


Uploading classifier/weights/xception_imagenet.pth


Uploaded classifier/weights/xception_imagenet.pth, 1 files out of an estimated total of 1


$AZUREML_DATAREFERENCE_182b99fdd3a743d2950a8c12c84befcc

In [9]:
from azureml.train.dnn import PyTorch

script_params = {
    '--data-folder': ds.as_mount(),
    '--epochs': 10,
    '--batch-size': 16,
    '--output_dir': './outputs'
}

estimator = PyTorch(source_directory=project_folder,
                    compute_target=compute_target,
                    entry_script='train_on_azure.py',
                    script_params=script_params,
                    node_count=2,
                    process_count_per_node=1,
                    distributed_backend='mpi',
                    use_gpu=True)

In [11]:
%%time
run = experiment.submit(estimator)
run.wait_for_completion(show_output=True)


RunId: pytorch-distr-hvd_1548151510988



Streaming azureml-logs/60_control_log_rank_0.txt



This is an MPI job. Rank:0
Streaming log file azureml-logs/60_control_log_rank_0.txt


Streaming log file azureml-logs/80_driver_log_rank_0.txt



Streaming azureml-logs/80_driver_log_rank_0.txt



...

Train Epoch: 3 [160/244 (62%)]	Loss: 0.465539




The experiment completed successfully. Finalizing run...


Cleaning up all outstanding Run operations, waiting 300.0 seconds
2 items cleaning up...
Cleanup took 0.20159578323364258 seconds



Execution Summary
RunId: pytorch-distr-hvd_1548151510988



CPU times: user 20.9 s, sys: 853 ms, total: 21.8 s
Wall time: 10min 53s


In [19]:
model = run.register_model(model_name='xception_1',
                           model_path='./outputs/xception_train_stdct.pth')
print(model.name, model.id, model.version, sep = '\t')

xception_1	xception_1:5	5


In [17]:
from azureml.widgets import RunDetails

RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [17]:
print(run)

Run(Experiment: pytorch-distr-hvd,
Id: pytorch-distr-hvd_1548095151019,
Type: azureml.scriptrun,
Status: Failed)


In [20]:
compute_target.delete()